In [ ]:
// Use TestContainers to start a postgres database
@file:DependsOn("org.testcontainers:junit-jupiter:1.21.3")
@file:DependsOn("org.testcontainers:postgresql:1.20.0")

In [ ]:
import org.testcontainers.containers.PostgreSQLContainer
import org.testcontainers.utility.DockerImageName
import org.testcontainers.utility.MountableFile

// Start the container with some preconfigured test data
val postgres = PostgreSQLContainer(DockerImageName.parse("postgres:16-alpine"))
    .withCopyToContainer(
        MountableFile.forHostPath("../src/test/resources/init_postgres.sql"),
        "/docker-entrypoint-initdb.d/init.sql"
    )
postgres.start()
DISPLAY("Is running: ${postgres.isRunning}")
DISPLAY("URL: ${postgres.jdbcUrl}")

In [ ]:
import org.jetbrains.kotlinx.jupyter.database.createDataSrc

// Variant 1: Create a simple data source:
val dataSrc1 = createDataSrc(
    jdbcUrl = postgres.jdbcUrl,
    username = "test",
    password = "test"
)
dataSrc1

In [ ]:
// Variant 2: Create a data source using Hikari:
val dataSrc2 = createDataSrc {
    jdbcUrl = postgres.jdbcUrl
    username = "test"
    password = "test"
    isReadOnly = true
}
dataSrc2

In [ ]:
import org.jetbrains.kotlinx.jupyter.database.createDataSrcFromSpring
import java.io.File
import kotlin.io.path.absolutePathString

val applicationFile = File("applications-test.properties").also {
    it.writeText(
        """
        spring.datasource.url=${postgres.jdbcUrl}
        spring.datasource.username=test
        spring.datasource.password=test
        """.trimIndent()
    )
}.toPath()

// Variant 3: Create a data source using Spring Application file. Both .properties and .yaml are supported.
val dataSrc3 = createDataSrcFromSpring(applicationFile)
dataSrc3

In [ ]:
import java.sql.Statement

// Use JDBC API's to read data
val conn = dataSrc1.connection
val statement = conn.createStatement()
val results = statement.executeQuery("SELECT * FROM users")
while (results.next()) {
    val id = results.getInt("id")
    val name = results.getString("name")
    println(id.toString() + ": " + name)
}


In [ ]:
%use dataframe

In [ ]:
// Load `users` table into DataFrame
dataSrc1.connection.use { connection ->
    val df = DataFrame.readSqlTable(connection, "users")
    df
}

In [ ]:
// Customize driver setup.
// For now, this is seperate API calls as the usage is expecte to be low.
// Also, they are cross cutting from creating data sources and adding them there will
// complicate those APIs.

// Add a custom driver
notebook.dataSourceDriverConfig.addDriver("postgres", "org.postgresql:postgresql:42.7.7")

// Use existing driver on the classpath
// notebook.dataSourceDriverConfig.clearUserDrivers()
notebook.dataSourceDriverConfig.useClasspathDriver("postgres")

In [ ]:
// Container must be manually stopped to prevent leaking it
postgres.stop()